In [1]:
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50 # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
import keras
from keras import Model
from keras.layers import Flatten, Dropout, Dense
from keras.optimizers import adadelta
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 建立 ResNet 模型
conv_base = ResNet50(input_shape=(32,32,3), include_top=False, weights='imagenet') 
conv_base.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [4]:
batch_size = 64
num_classes = 10
epochs = 30

x = conv_base.output
x = Flatten()(x)
x = Dropout(0.5)(x)

output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

model = Model(inputs=conv_base.input, outputs=output_layer)

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train, 
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=1),
                     keras.callbacks.ModelCheckpoint("best_model.h5", monitor='val_loss')],
          verbose=1,
          validation_data=(x_test, y_test))

model = keras.models.load_model("best_model.h5")
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 219s 4ms/step - loss: 1.7535 - accuracy: 0.5471 - val_loss: 2.2091 - val_accuracy: 0.2938
Epoch 2/30
50000/50000 [==============================] - 197s 4ms/step - loss: 0.9931 - accuracy: 0.6905 - val_loss: 1.4232 - val_accuracy: 0.6226
Epoch 3/30
50000/50000 [==============================] - 197s 4ms/step - loss: 0.7892 - accuracy: 0.7506 - val_loss: 1.5642 - val_accuracy: 0.5514
Test loss: 1.564192866706848
Test accuracy: 0.5514000058174133
